In [1]:
import numpy as np
from scipy.optimize import curve_fit, root_scalar, minimize

In [2]:
########################################################################################
#      Import Tunable Transmon Parameters
########################################################################################

class Transmon:
    def __init__(self, transmon_popt, name='Qubit'):
        '''
        transmon_popt: [x0, a, fq, ec, d]
        '''
        x0, a, fq, ec, d = transmon_popt
        self.name = name
        self.transmon_popt = transmon_popt
        self.flux_quantum_voltage = fq
        self.voltage_offset       = x0

        # flux -> freq
        self.freq = create_qubit_function(self.transmon_popt)     
        # freq -> flux
        self.flux = create_qubit_inverse_function(self.freq) 

        # functions using V, no crosstalk correction
        self.V_to_freq = lambda V: transmon_fit(V, *self.transmon_popt)
        self.flux_to_V = create_qubit_flux_to_voltage(self.transmon_popt)
        self.V_to_flux = create_qubit_voltage_to_flux(self.transmon_popt)

def transmon_fit(x, x0, a, fq, ec, d):
    '''
        x0: offset (Volts)
        a: maximum qubit frequency
        fq: flux quantum in volts
        ec: "charging energy", offset
        d: junction asymmetry
    '''
    return np.sqrt(a**2 *np.sqrt(np.cos(np.pi/fq*(x-x0))**2 + (d**2)*np.sin(np.pi/fq*(x-x0))**2)) - ec

def coupler_resonator_fit(x, x0, a, fq, ec, d, β, Ω, M):
     '''a: maximum qubit frequency ~ 5
       fq: flux quantum in volts ~ 2 to 3
       ec: "charging energy" ~ 0.2
       d: junction asymmetry E [0, 1.0]
       β: coupling factor ~ 1e-3
       Ω: resonator frequency ~ 7 (average of your resonator)
       M: crosstalk factor ~ 1e-5
    '''
     
     coupler_freq = transmon_fit(x, x0, a, fq, ec, d)
     resonator_freq = Ω + M * x # effect of other qubits' crosstalk
    
     g = β * np.sqrt(coupler_freq*resonator_freq)
     delta = coupler_freq - resonator_freq
     return resonator_freq - g**2 / delta

# Transmon_BOUNDS = (
#      # x0, a, fq, ec, d
#      [-np.inf, 1., 1., 0.1, 0.],
#      [np.inf, 10., 5., 0.3, 1.])

# coupler_resonator_bounds = (
#      # x0, a, fq, ec, d, β, Ω, M
#      [-np.inf, 1., 1., 0.1, 0., 0., 5., 0.,],
#      [np.inf, 10., 5., 0.3, 1., np.inf, 10., np.inf])

def Q_deriv(x, a,d,c):
    '''First derivative of transmon_fit with respect to flux'''
    x = x*np.pi
    return 0.25 * a * (-2 * np.cos(x) * np.sin(x) + 2 * (d ** 2) * np.cos(x) * np.sin(x)) * ((a * (np.sqrt(np.cos(x) ** 2 + (d ** 2) * (np.sin(x) ** 2)))) ** -0.5) * ((np.cos(x) ** 2 + (d ** 2) * (np.sin(x) ** 2)) ** -0.5)

def create_qubit_function(_popt):
        x0, a, fq, ec, d = _popt
        return lambda x: transmon_fit(fq*x + x0, *_popt)

def create_qubit_inverse_function(qubit_function):
        def find_root(f, __qubit_function):
            bracket = (0, 0.5)

            if isinstance(f, (list, np.ndarray)):
                fluxes = np.empty(len(f))
                for i in range(len(f)):
                    root_function = lambda flux: __qubit_function(flux) - f[i]
                    result = root_scalar(root_function, bracket=bracket)
                    fluxes[i] = result.root
                return fluxes
            elif isinstance(f, (int, float)):
                root_function = lambda flux: __qubit_function(flux) - f
                result = root_scalar(root_function, bracket=bracket)
                return result.root
        return lambda f: find_root(f, qubit_function)

def create_qubit_flux_to_voltage(_popt):
        x0, a, fq, ec, d = _popt
        return lambda x: fq*x + x0
    
def create_qubit_voltage_to_flux(_popt):
        x0, a, fq, ec, d = _popt
        return lambda x: (x - x0)/fq

In [3]:
qubit_params_dir = r'Z:\QSimMeasurements\Measurements\8QV1_Triangle_Lattice\qubit_parameters'

In [4]:
C1 = Transmon(np.load(fr'{qubit_params_dir}\C1_popt.npy'), 'C1')
C2 = Transmon(np.load(fr'{qubit_params_dir}\C2_popt.npy'), 'C2')
C3 = Transmon(np.load(fr'{qubit_params_dir}\C3_popt.npy'), 'C3')
C4 = Transmon(np.load(fr'{qubit_params_dir}\C4_popt.npy'), 'C4')
C5 = Transmon(np.load(fr'{qubit_params_dir}\C5_popt.npy'), 'C5')
C6 = Transmon(np.load(fr'{qubit_params_dir}\C6_popt.npy'), 'C6')

FileNotFoundError: [Errno 2] No such file or directory: 'Z:\\QSimMeasurements\\Measurements\\8QV1_Triangle_Lattice\\qubit_parameters\\C1_popt.npy'